In [ ]:
!wget -O wheelchair.zip https://app.roboflow.com/ds/th2kY31t7e?key=gB9mY7qh61

In [ ]:
import zipfile

with zipfile.ZipFile('/content/wheelchair.zip') as target_file:
  target_file.extractall('/content/wheelchair/')

In [ ]:
!cat /content/wheelchair/data.yaml

In [ ]:
!pip install PyYAML

In [ ]:
import yaml

data = {
    'train' : '/content/wheelchair/train/images/',
    'val' : '/content/wheelchair/valid/images/',
    'test' : '/content/wheelchair/test/images/',
    'names' : ['Wheel', 'Wheel-chairs', 'handicap-people', 'people_wheelchair', 'wheel chair', 'wheelchair'],
    'nc' : 6
}

with open('/content/wheelchair/wheelchair_Data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('/content/wheelchair/wheelchair_Data.yaml', 'r') as f:
  wheelchair_yaml = yaml.safe_load(f)
  display(wheelchair_yaml)

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics


ultralytics.checks()

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [ ]:
print(type(model.names), len(model.names))
print(model.names)

In [ ]:
!mkdir /content/wheelchair/valid
!mkdir /content/wheelchair/test

In [ ]:
from glob import glob

img_list = glob('/content/wheelchair/train/images/*.jpg')

print(len(img_list))

In [ ]:
!mkdir /content/wheelchair/extra

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2023)

test_img_list, val_img_list = train_test_split(val_img_list, test_size=0.5, random_state=2023)

print(len(train_img_list), len(val_img_list), len(test_img_list))

In [ ]:
with open('/content/wheelchair/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/wheelchair/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

with open('/content/wheelchair/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

In [ ]:
import yaml

with open('/content/wheelchair/wheelchair_Data.yaml', 'r') as f:
  wheelchair_yaml = yaml.safe_load(f)

print(wheelchair_yaml)

wheelchair_yaml['train'] = '/content/wheelchair/train.txt'
wheelchair_yaml['val'] = '/content/wheelchair/val.txt'
wheelchair_yaml['test'] = '/content/wheelchair/test.txt'

with open('/content/wheelchair/wheelchair_Data.yaml', 'w') as f:
  yaml.dump(wheelchair_yaml, f)

print(wheelchair_yaml)

In [ ]:
model.train(data='/content/wheelchair/wheelchair_Data.yaml', epochs=100, patience=30, batch=32, imgsz=416)

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

In [ ]:
result = model.predict(source='/content/wheelchair/test.txt/', save=True)

In [ ]:
from google.colab.patches import cv2_imshow # google의 자체 개발 라이브러리에서 cv2_imshow를 import

res_plotted = result[0].plot() # plot() 함수를 이용해서 이미지 내에 bounding box나 mask 등의 result 결과를 그릴 수 O
# print(res_plotted)

# plt.figure(figsize=(12, 12))
# plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
# plt.show()

cv2_imshow(res_plotted)

In [ ]:
from google.colab import files
import cv2
import numpy as np

# 파일 업로드
uploaded = files.upload()

# 업로드한 파일의 이름 가져오기
file_name = list(uploaded.keys())[0]

# OpenCV로 이미지 읽기
image = cv2.imread(file_name)

# 모델에 적용 (예: YOLOv8 모델 사용)
results = model(image)  # YOLO 모델 추론
res_plotted = results[0].plot()

# 결과 표시
from google.colab.patches import cv2_imshow
cv2_imshow(res_plotted)
